In [1]:
using JSON
using Distributed
using Plots
using LaTeXStrings
using ProgressMeter
theme(:default)
if nprocs() < length(Sys.cpu_info()) - 50
    addprocs(length(Sys.cpu_info()) - 50 - nprocs())
end
println("Using ",nprocs()," processes.")
include("./fermionise.jl")
include("./models.jl")
include("./quantuminfo.jl")

Using 46 processes.


MutualInfo (generic function with 1 method)

In [ ]:
function main(numBath)
    oneparthop_d0_arr = []
    impBathSflip_arr = []
    impBathCflip_arr = []
    oneparthop_01_arr = []
    basisStates = BasisStates(2 * (1 + numBath), 1 + numBath)
    U_arr = 1:0.5:20
    couplingsArray = [(1, 2, U, 1, -U, 0) for U in U_arr]
    computables = [
        ["hh" (1,2)]; 
        ["+-+-" (1,2,4,3,)]; 
        ["+-+-" (2, 1, 3, 4)]; 
        ["++--" (1,2,4,3,)]; 
        ["--++" (2, 1, 3, 4)]; 
        ["+-" (3,7)]; 
        ["+-" (4,8)];
        ]
    (imp_hh, d0_spm, d0_smp, d0_cpm, d0_cmp, hop_01up, hop_01dn) = ComputeCorrelations(basisStates, TOSHamiltonianRSpace, couplingsArray, computables)
    p = plot(palette = :Set1_9, thickness_scaling = 1.2, fontfamily="Roboto-Regular")
    plot!(U_arr, imp_hh, lw=3, label="h-occupancy")
    plot!(U_arr, hop_01up .+ hop_01dn, lw=3, label=L"\langle c_0^\dagger c_1\rangle")
    plot!(U_arr, abs.(d0_spm .+ d0_smp), lw=3, label=L"\langle S_d^+S_0^-\rangle")
    plot!(U_arr, d0_cpm .+ d0_cmp, lw=3, label=L"\langle C_d^+ C_0^-\rangle")
    xlabel!(L"U/t_\perp")
    ylabel!("correlations")
    display(p)
    savefig(p, "myplot.pdf")
end

println(main(6))